In [5]:
import numpy as np
import pandas as pd

In [7]:
진로변경방법위반v1 = pd.read_excel('data/진로변경방법위반_v1.xlsx')
진로변경방법위반v1['주소'].value_counts() 

주소
부산광역시 사하구 하단동 491번길        237
경기도 파주시 목동동 951번길          153
경상북도 안동시 풍천면 갈전리 1735번길    146
경기도 파주시 목동동 1150번길         144
서울특별시 관악구 봉천동 1693번길       142
                          ... 
경기도 하남시 창우동 15번길             1
경상북도 경주시 건천읍 건천리 597번길       1
대전광역시 유성구 갑동 437번길           1
경기도 파주시 문산읍 내포리 89번길         1
경상남도 고성군 고성읍 기월리 80번길        1
Name: count, Length: 31682, dtype: int64

In [3]:
중앙선침범v1[['위도', '경도']].value_counts().to_frame().head(20)

,,count
위도,경도,
37.544349,127.175337,31
35.122779,126.861528,14
35.834433,127.132690,14
37.465706,127.136387,14
35.143482,126.920704,10
35.152395,126.896504,8
35.159369,126.922020,8
35.124234,126.932387,7
34.927214,128.426854,7


In [2]:
excel = pd.read_excel('data/신호위반.xlsx')

주소
부산광역시 사하구 하단동 491번길        237
경기도 파주시 목동동 951번길          153
경상북도 안동시 풍천면 갈전리 1735번길    146
경기도 파주시 목동동 1150번길         144
서울특별시 관악구 봉천동 1693번길       142
                          ... 
경기도 하남시 창우동 15번길             1
경상북도 경주시 건천읍 건천리 597번길       1
대전광역시 유성구 갑동 437번길           1
경기도 파주시 문산읍 내포리 89번길         1
경상남도 고성군 고성읍 기월리 80번길        1
Name: count, Length: 31682, dtype: int64

In [9]:
excel.head()

,발생일,위도,경도
0,2022-12-31,37.564914,126.927373
1,2022-12-31,35.511526,129.424473
2,2022-12-31,37.396434,126.961077
3,2022-12-31,37.496975,126.883361
4,2022-12-31,34.759205,127.710926


In [10]:
import openpyxl
import requests
import pandas as pd
import time

# excel 파일 열기
wb = openpyxl.load_workbook('data/신호위반.xlsx')
sheet = wb.active

# 위도와 경도 읽어오기 및 결과 저장 리스트 생성
latitude = []
longitude = []
addresses = []

for row in sheet.iter_rows(min_row=2):
    if row[0].value and row[1].value and row[2].value:
        latitude.append(row[1].value)
        longitude.append(row[2].value)

# 주소 요청 함수 정의 및 각각의 위도와 경도에 대해 주소 요청하기 
def get_address(latitude, longitude):
    url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords={longitude},{latitude}&sourcecrs=epsg:4326&output=json&orders=addr,admcode,roadaddr"
    response = requests.get(url, headers={
        "X-NCP-APIGW-API-KEY-ID": "bzire4m1dj",
        "X-NCP-APIGW-API-KEY": "kXwlouhxbtWX0zNhYUBjA6SxFz8dq1aFQpY0l0hr",
    })
    return response.json()

for lat, lon in zip(latitude, longitude):
    try:
        result = get_address(lat, lon)
        
        # 결과가 OK인 경우에만 처리하기 
        if result['status']['name'] == 'ok':
            road_address_name = result['results'][0]['region']['area1']['name'] + ' ' + \
                result['results'][0]['region']['area2']['name'] + ' ' + \
                result['results'][0]['region']['area3']['name'] + ' ' + \
                result['results'][0]['region']['area4']['name']

    # 도로명 주소가 있는 경우 추가하기 
            if 'road_address' in result['results'][0]:
                road_address_name += ' ' + result['results'][0]['road_address']['name']

            # 지번주소 사용하기 
            else:
                jibun_address_name = road_address_name
                if 'land' in result['results'][0]:
                    jibun_address_name += f" {result['results'][0]['land']['number1']}번길"
                    if result['results'][0]['land'].get('number2'):
                        jibun_address_name += f"-{result['results'][0]['land'].get('number2')}"
                
                addresses.append(jibun_address_name)

        else:
            addresses.append(road_address_name)

    except Exception as e:
        print(f"An error occurred: {e}")
        addresses.append('')
    
    time.sleep(0.1)  # API 호출 간에 약간의 지연 시간 추가

# 받아온 주소를 excel 파일에 저장하기
for i in range(len(addresses)):
    sheet.cell(row=i+2, column=4).value = addresses[i]

# excel 파일 저장하기
wb.save('data/신호위반_result.xlsx')

df = pd.read_excel('data/신호위반_result.xlsx')
df['Address'] = addresses  # Address라는 새 열을 만들고 그곳에 addresses 리스트의 값을 넣습니다.
df.to_excel('data/신호위반_result.xlsx', index=False)